In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from LaneInput import LaneInput  # Class to input lane regions
from PSMorg import PEDESTRIANESTIMATOR # Updated SpeedEstimator2 class with time increment
from time_utils import increment_time, get_time_from_xml  # Import utility functions

# Initialize the YOLO model for object detection
model = YOLO("best(3).pt")  # Replace with the path to your YOLO model
names = model.model.names  # List of class names for the YOLO model

# Initialize video capture
video_path = "C0148.mp4"  # Replace with the path to your video
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), f"Error reading video file {video_path}"

# Get video properties (width, height, and FPS)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, 
                                      cv2.CAP_PROP_FRAME_HEIGHT, 
                                      cv2.CAP_PROP_FPS))

# Video writer to save the processed video with lane detection
output_video = "lane_detection_output.avi"
video_writer = cv2.VideoWriter(output_video, 
                               cv2.VideoWriter_fourcc(*"mp4v"), 
                               fps, (w, h))

# Initialize LaneInput to capture lane points
lane_input = LaneInput()

# Capture the first frame from the video to input lane points
success, first_frame = cap.read()
if not success:
    raise ValueError("Failed to read the first frame from the video.")

# Input the lane regions in the first frame
lane_input.input_lanes(first_frame)
lane_regions = lane_input.get_lanes()  # Get the captured lane regions
if not lane_regions:
    raise ValueError("No lane regions captured. Please ensure lanes are defined.")

print("Captured Lane Regions:", lane_regions)

# Load initial time from the XML file
xml_file = r"E:\learn phython\speed 2\Speed-detection-of-vehicles\C0148M01.XML"
current_time = get_time_from_xml(xml_file)

# Initialize SpeedEstimator2
lane_estimator = SpeedEstimator2(
    names=names,
    lane_regions=lane_regions,
    view_img=True,  # Display image flag
    target_x=500,   # Define the X-coordinate target for lane estimation
    csv_file="lane_log.csv"  # Define the CSV file for logging
)

# Frame processing
frame_counter = 0
skip_frames = 10  # Number of frames to skip after processing one frame

# Main loop for processing each video frame
while cap.isOpened():
    success, im0 = cap.read()  # Read the next frame
    if not success:
        print("Video processing completed or frame not available.")
        break

    # Process every (skip_frames + 1)-th frame
    if frame_counter % (skip_frames + 1) == 0:
        # Perform object detection and tracking using the YOLO model
        tracks = model.track(im0, persist=True)
        
        # Increment the current time based on the number of frames skipped
        current_time = increment_time(current_time, skip_frames + 1, fps)
        
        # Pass the tracks and the frame to SpeedEstimator2's check_and_log_lanes function
        im0 = lane_estimator.process_frame(im0, tracks, current_time)

        # Display the processed frame with lane estimation
        cv2.imshow("Live Output", im0)

        # Write the processed frame to the output video
        video_writer.write(im0)

    # Increment frame counter
    frame_counter += 1

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Processing terminated by user.")
        break

# Release video capture and writer resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processing complete. Output saved to {output_video}")


Please input the number of lanes:


In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from LaneInput import LaneInput  # Class to input lane regions
from PSMorg import SpeedEstimator2  # Updated SpeedEstimator2 class with time increment
from time_utils import increment_time, get_time_from_xml  # Import utility functions

# Initialize the YOLO model for object detection
model = YOLO("best(3).pt")  # Replace with the path to your YOLO model
names = model.model.names  # List of class names for the YOLO model

# Initialize video capture
video_path = "export6.mp4"  # Replace with the path to your video
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), f"Error reading video file {video_path}"

# Get video properties (width, height, and FPS)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, 
                                      cv2.CAP_PROP_FRAME_HEIGHT, 
                                      cv2.CAP_PROP_FPS))

# Video writer to save the processed video with lane detection
output_video = "lane_detection_output.avi"
video_writer = cv2.VideoWriter(output_video, 
                               cv2.VideoWriter_fourcc(*"mp4v"), 
                               fps, (w, h))

# Capture the first frame from the video to input lane points
success, first_frame = cap.read()  # Read the first frame from the video
#if not success or first_frame is None:
    #raise ValueError("Failed to read the first frame from the video.")

# Initialize LaneInput to capture lane points
lane_input = LaneInput()

# Input the lane regions in the first frame
#lane_input.input_lanes(first_frame)
lane_regions, lane_centers = lane_input.get_lanes_and_centers()  # Get the captured lane regions and centers
if not lane_regions:
    #raise ValueError("No lane regions captured. Please ensure lanes are defined.")

 print("Captured Lane Regions:", lane_regions)

# Load initial time from the XML file
xml_file = r"E:\learn phython\speed 2\Speed-detection-of-vehicles\C0148M01.XML"
current_time = get_time_from_xml(xml_file)

# Initialize SpeedEstimator2
lane_estimator = SpeedEstimator2(
    names=names,
    lane_regions=lane_regions,
    lane_centers=lane_centers,  # Pass the lane centers as well
    view_img=True,  # Display image flag
    target_x=500,   # Define the X-coordinate target for lane estimation
    csv_file="lane_log.csv"  # Define the CSV file for logging
)

# Frame processing
frame_counter = 0
skip_frames = 10  # Number of frames to skip after processing one frame

# Main loop for processing each video frame
while cap.isOpened():
    success, im0 = cap.read()  # Read the next frame
    if not success:
        print("Video processing completed or frame not available.")
        break

    # Process every (skip_frames + 1)-th frame
    if frame_counter % (skip_frames + 1) == 0:
        # Perform object detection and tracking using the YOLO model
        tracks = model.track(im0, persist=True)
        
        # Increment the current time based on the number of frames skipped
        current_time = increment_time(current_time, skip_frames + 1, fps)
        
        # Pass the tracks and the frame to SpeedEstimator2's check_and_log_lanes function
        im0 = lane_estimator.process_frame(im0, tracks, current_time)

        # Display the processed frame with lane estimation
        cv2.imshow("Live Output", im0)

        # Write the processed frame to the output video
        video_writer.write(im0)

    # Increment frame counter
    frame_counter += 1

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Processing terminated by user.")
        break

# Release video capture and writer resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processing complete. Output saved to {output_video}")


[(318, 427), (687, 417), (1081, 389)]

0: 384x640 6 motobikes, 2 pedestrians, 8 sedans, 650.5ms
Speed: 0.0ms preprocess, 650.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 motobikes, 2 pedestrians, 7 sedans, 630.4ms
Speed: 17.7ms preprocess, 630.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motobikes, 4 pedestrians, 7 sedans, 612.9ms
Speed: 0.0ms preprocess, 612.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 motobike, 5 pedestrians, 6 sedans, 599.3ms
Speed: 0.0ms preprocess, 599.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 motobikes, 3 pedestrians, 5 sedans, 630.5ms
Speed: 0.0ms preprocess, 630.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 pedestrians, 5 sedans, 654.5ms
Speed: 0.0ms preprocess, 654.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 motobike, 4 pedestrians, 